**This is Food Ingredients recipe RNN model, that gives the reciepy for the given food ingredients**

# **RNN**
A Recurrent Neural Network (RNN) is a type of neural network designed for processing sequential data. Unlike traditional feedforward networks, which process inputs independently, RNNs maintain a memory of previous inputs by using loops in their architecture.

This makes them well-suited for tasks where context and order matter, such as time series forecasting, speech recognition and text generation

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pes12017000148/food-ingredients-and-recipe-dataset-with-images")

print("Path to dataset files:", path)

100%|██████████| 206M/206M [00:05<00:00, 39.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/pes12017000148/food-ingredients-and-recipe-dataset-with-images/versions/1


In [3]:
!mv /root/.cache/kagglehub/datasets/pes12017000148/food-ingredients-and-recipe-dataset-with-images/versions/1 /content

In [1]:
import pandas as pd

df = pd.read_csv("/content/1/Food Ingredients and Recipe Dataset with Image Name Mapping.csv")
df.head()

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [2]:
df['Cleaned_Ingredients'][0]

'[\'1 (3½–4-lb.) whole chicken\', \'2¾ tsp. kosher salt, divided, plus more\', \'2 small acorn squash (about 3 lb. total)\', \'2 Tbsp. finely chopped sage\', \'1 Tbsp. finely chopped rosemary\', \'6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature\', \'¼ tsp. ground allspice\', \'Pinch of crushed red pepper flakes\', \'Freshly ground black pepper\', \'⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)\', \'2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces\', \'2 Tbsp. extra-virgin olive oil\', \'½ small red onion, thinly sliced\', \'3 Tbsp. apple cider vinegar\', \'1 Tbsp. white miso\', \'¼ cup all-purpose flour\', \'2 Tbsp. unsalted butter, room temperature\', \'¼ cup dry white wine\', \'2 cups unsalted chicken broth\', \'2 tsp. white miso\', \'Kosher salt\', \'freshly ground pepper\']'

## Data Cleaning

In [3]:
import re
import ast

stoplist = {
    'tsp', 'teaspoon', 'tbsp', 'tablespoon', 'cup', 'cups', 'oz', 'lb', 'pound', 'pounds', 'inch', 'inches', 'pinch',
    'loaf', 'piece', 'pieces', 'sprig', 'sprigs', 'clove', 'cloves', 'head', 'heads', 'bunch', 'bunches', 'can', 'cans',
    'jar', 'jars', 'package', 'packages', 'bottle', 'bottles', 'slice', 'slices', 'strip', 'strips', 'sheet', 'sheets',
    'block', 'blocks', 'stick', 'sticks', 'cube', 'cubes', 'dash', 'drop', 'drops', 'handful', 'handfuls', 'scoop',
    'scoops', 'splash', 'splashes', 'wedge', 'wedges', 'zest', 'zests', 'grated', 'minced', 'chopped', 'sliced', 'diced',
    'peeled', 'seeded', 'halved', 'quartered', 'crushed', 'ground', 'freshly', 'divided', 'plus', 'more', 'room',
    'temperature', 'optional', 'for', 'serving', 'to', 'taste', 'small', 'medium', 'large', 'whole', 'bone-in', 'skinless',
    'boneless', 'skin-on', 'fresh', 'dried', 'canned', 'storebought', 'homemade', 'about', 'lightly', 'packed', 'coarsely',
    'finely', 'thinly', 'thickly', 'preferably', 'such', 'as', 'like', 'e.g.', 'i.e.', 'or', 'and', 'with', 'without',
    'including', 'excluding', 'total', 'approx', 'approximately', 'etc', 'of', 'into', 'cut', 'torn', 'melted', 'cored',
    'seeded', 'peeled', 'drained', 'rinsed', 'trimmed', 'scrubbed', 'halved', 'quartered', 'sliced', 'diced', 'chopped',
    'minced', 'grated', 'crushed', 'ground', 'freshly', 'stems', 'removed','-inch','-qt','flat-leaf','round', 'Italian','teaspoons',
    'tablespoon', 'sage','ozg','lbg','in','allspice','cupg','cupskg','cupsml','cupml','tbspml', 'any', 'combination','fruit',
    'Zatarains', 'recipe', 'ounces', '-ounce', 'ounce'
}

all_items = set()

# Function to check if a word is a number, fraction, or single character
def is_number_fraction_or_single_char(word):
    # Check if word is a single character
    if len(word) == 1:
        return True
    # Check if word is numeric
    if word.isdigit():
        return True
    # Check if word contains fraction characters
    if any(char in '½¾⅓⅔¼⅛' for char in word):
        return True
    return False

# Function to clean a single ingredient string
def clean_ingredient(ingredient):
    # Remove brackets and their contents
    ingredient = re.sub(r'\[.*?\]|\(.*?\)', '', ingredient)
    # Remove all characters that are not letters, spaces, or hyphens
    ingredient = re.sub(r'[^a-zA-Z\s-]', '', ingredient)
    # Split into words
    words = ingredient.split()
    # Filter out stop words, numbers, fractions, and single characters
    filtered_words = [word for word in words if word.lower() not in stoplist and not is_number_fraction_or_single_char(word)]
    # Join back into a string
    cleaned = ' '.join(filtered_words)
    # Return None if empty
    return cleaned if cleaned else None

# Function to process a list of ingredients
def clean_ingredient_list(ingredient_list):
    # If input is a string, safely evaluate it to a list
    if isinstance(ingredient_list, str):
        try:
            ingredient_list = ast.literal_eval(ingredient_list)
        except (ValueError, SyntaxError):
            return []
    # Clean each ingredient and filter out None values
    items = [cleaned for cleaned in (clean_ingredient(ing) for ing in ingredient_list) if cleaned]
    all_items.update(items)
    # Return a comma-separated string
    return ",".join(items)


data = df['Cleaned_Ingredients'][200].split(", ")
cleaned = clean_ingredient_list(data)

cleaned


'sugar,water,watermelon,cantaloupe,pineapple,papaya,strawberries,oranges,peaches,mangoes,Ice,Lemon lime'

In [4]:
df['Cleaned_Item_Names'] = df['Cleaned_Ingredients'].apply(clean_ingredient_list)

In [5]:
df.drop(["Cleaned_Ingredients","Unnamed: 0"], axis=1, inplace=True)

In [6]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [7]:
df.head()

,Title,Ingredients,Instructions,Image_Name,Cleaned_Item_Names
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"chicken,kosher salt,acorn squash,rosemary,unsa..."
1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"egg whites,new potatoes,kosher salt,black pepp..."
2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"evaporated milk,milk,garlic powder,onion powde..."
3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"Italian,tablespoons olive oil,sweet Italian sa..."
4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"dark brown sugar,hot water,bourbon,lemon juice..."


In [8]:
all_items_list = list(all_items)
all_items_list[:10]

['creamy peanut butter',
 'sweetened whipped cream',
 'Bosc pears -inch-thick',
 'bacon squares',
 '-inch-diameter golden beets healthy greens beets each greens -inch-wide',
 'peppermint extract',
 '-pound beef tenderloin well',
 '-pound chickens lengthwise patted dry',
 'thick piquillo peppers roasted red peppers from',
 'hardwood chips']

## Data Preprocessing

In [9]:
df['ID'] = range(0, len(df))

In [10]:
df.head()

,Title,Ingredients,Instructions,Image_Name,Cleaned_Item_Names,ID
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"chicken,kosher salt,acorn squash,rosemary,unsa...",0
1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"egg whites,new potatoes,kosher salt,black pepp...",1
2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"evaporated milk,milk,garlic powder,onion powde...",2
3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"Italian,tablespoons olive oil,sweet Italian sa...",3
4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"dark brown sugar,hot water,bourbon,lemon juice...",4


In [11]:
# tokenize
def tokenize(text):
    text = text.lower()
    tokens = []
    for part in text.split(','):
        tokens.extend(part.split())
    return tokens

In [12]:
tokenize(df['Cleaned_Item_Names'][200])

['sugar', 'water', 'ice', 'lemon', 'lime']

In [13]:
# vocab
vocab = {'<UNK>':0}

In [14]:
def build_vocab(row):
    tokenized_row = tokenize(row)
    for token in tokenized_row:
        if token not in vocab:
            vocab[token] = len(vocab)

In [15]:
df['Cleaned_Item_Names'].apply(build_vocab)
len(vocab)

7072

In [16]:
# tokenize words to numerical indices
def text_to_indices(text, vocab):
    indexed_text = []
    for token in tokenize(text):
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])
    return indexed_text

## Data Loading

In [17]:
import torch
from torch.utils.data import Dataset, DataLoader

In [18]:
class IngredientsDataset(Dataset):
    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        numerical_ingredients = text_to_indices(self.df['Cleaned_Item_Names'].iloc[idx], self.vocab)
        numerical_dish = self.df['ID'].iloc[idx]
        return torch.tensor(numerical_ingredients, dtype=torch.long), torch.tensor(numerical_dish, dtype=torch.long)


In [19]:
dataset = IngredientsDataset(df, vocab)

In [20]:
dataset[0]

(tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 10, 13, 14, 15, 16, 17,
         18, 19, 20,  9, 21, 22, 23, 24, 15, 25, 26, 27,  7,  8, 28, 15, 29,  7,
          1, 30, 15, 25,  2,  3, 10]),
 tensor(0))

In [21]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    texts, labels = zip(*batch)
    padded_texts = pad_sequence(texts, batch_first=True, padding_value=0)
    return padded_texts, torch.stack(labels)

In [22]:
dataloader = DataLoader(dataset, batch_size=32, collate_fn=collate_fn, shuffle=True, pin_memory=True)

## Model

In [23]:
import torch.nn as nn

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device:",device)

using device: cuda


In [25]:
class IngredientsRNN(nn.Module):
    def __init__(self, vocab_size, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
        self.rnn = nn.RNN(50, 64, batch_first=True)

        self.fc = nn.Linear(64, num_classes)

    def forward(self, ingredients):
        embedded = self.embedding(ingredients)                       # (batch_size, seq_len, 50)
        hidden, _ = self.rnn(embedded)                               # (batch_size, seq_len, 64)
        last_hidden = hidden[:, -1, :]                               # (batch_size, 64)
        output = self.fc(last_hidden)                                # (batch_size, vocab_size)
        return output

In [26]:
learning_rate = 0.0001
epochs = 20

In [27]:
num_classes = df['ID'].nunique()

In [28]:
model = IngredientsRNN(len(vocab), num_classes).to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
# training loop
for epoch in range(epochs):
    total_loss = 0
    for ingredients, labels in dataloader:
        ingredients = ingredients.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # forward pass
        output = model(ingredients)

        # loss
        loss = loss_function(output, labels)

        # back prop
        loss.backward()

        # update
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch: {epoch+1}, Loss: {total_loss/len(dataloader)}")

Epoch: 1, Loss: 9.562265622107338
Epoch: 2, Loss: 9.517507797168895
Epoch: 3, Loss: 9.515052201058626
Epoch: 4, Loss: 9.513130377819188
Epoch: 5, Loss: 9.512443861125204
Epoch: 6, Loss: 9.510432819619563
Epoch: 7, Loss: 9.509827600271215
Epoch: 8, Loss: 9.508330869448693
Epoch: 9, Loss: 9.507395373701485
Epoch: 10, Loss: 9.505241606472794
Epoch: 11, Loss: 9.50433792548157
Epoch: 12, Loss: 9.503478983567224
Epoch: 13, Loss: 9.502446321514546
Epoch: 14, Loss: 9.501116289346704
Epoch: 15, Loss: 9.499479162749521
Epoch: 16, Loss: 9.498637988104074
Epoch: 17, Loss: 9.497064509098
Epoch: 18, Loss: 9.495778947079916
Epoch: 19, Loss: 9.494330767771643
Epoch: 20, Loss: 9.493984629192623


In [58]:
def predict(model, question, threshold=0.5):
    numerical_ingredients = text_to_indices(question, vocab)
    tensor_ingredients = torch.tensor(numerical_ingredients, dtype=torch.long).unsqueeze(0).to(device)
    output = model(tensor_ingredients)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    value, index = torch.max(probabilities, dim=1)



    return value, index


In [68]:
# Predict dish based on input ingredients
ingredients = "green onions,tablespoons sugar,garlic,ginger thin rounds,soy sauce,mirin,Asian sesame oil,tablespoons toasted sesame seeds,black pepper,boned butterflied leg lamb excess fat,Asian pear,green onions,soy sauce,mirin,tablespoons sugar,tablespoons Asian sesame oil,tablespoons toasted sesame seeds,Nonstick vegetable oil spray,jalapeo chiles,garlic,kochujang,butter lettuce leaves separated left,Ingredient info Mirin is sold the Asian foods section some supermarkets at Japanese markets Kochujang is spicy red paste made pureed fermented soybeans hot chiles Its available at Korean markets online from koamartcom If you cant find Kochujang substitute miso mixed tablespoons sriracha hot sauce Test-Kitchen tip Ask your butcher bone butterfly trim the leg lamb you Save the bone pop it the freezer use it make soup stock,metal skewers bamboo skewers soaked water at least hour"
value, index = predict(model, ingredients)

# Print prediction confidence
print(f"Confidence: {value.item() * 100:.2f}%")

# Move index to CPU and convert to integer
index = index.cpu().item()

# Retrieve corresponding dish from DataFrame
row = df[df["ID"] == index]

# Display dish information
if not row.empty:
    print(f"\nTitle: {row['Title'].values[0]}")
    print(f"Ingredients: {row['Ingredients'].values[0]}")
    print(f"Instructions: {row['Instructions'].values[0]}")
    print(f"Image Name: {row['Image_Name'].values[0]}")
    print(f"Cleaned Item Names: {row['Cleaned_Item_Names'].values[0]}")
else:
    print("ID not found.")


Confidence: 0.04%

Title: Lamb Bulgogi with Asian Pear Dipping Sauce
Ingredients: ['4 green onions, coarsely chopped', '3 tablespoons sugar', '3 garlic cloves, coarsely chopped', '1 2-inch piece fresh ginger, peeled, cut into thin rounds', '2/3 cup soy sauce', '2/3 cup mirin (sweet Japanese rice wine)', '1/3 cup Asian sesame oil', '2 tablespoons toasted sesame seeds', '1 teaspoon freshly ground black pepper', '1 boned butterflied leg of lamb (about 5 1/2 pounds; from one 6 1/2- to 7-pound bone-in leg), trimmed of excess fat', '1 cup chopped peeled cored Asian pear (about 1/2 large)', '10 green onions; 2 chopped, 8 trimmed', '1/2 cup soy sauce', '1/2 cup mirin (sweet Japanese rice wine)', '3 tablespoons sugar', '3 tablespoons Asian sesame oil', '4 tablespoons toasted sesame seeds, divided', 'Nonstick vegetable oil spray', '8 jalapeño chiles, halved (seeded, if desired)', '8 garlic cloves, peeled', '1/2 cup kochujang (Korean hot pepper paste)', '1 large head of butter lettuce, leaves sep